In [73]:
import pandas as pd
import re

In [74]:
train_df = pd.read_csv("../data/raw/train.csv")
test_df = pd.read_csv("../data/raw/test.csv")

In [75]:
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


## Handle Missing Data

In [76]:
for column in train_df.columns:
    if train_df[column].isna().sum() > 0:
        print(column)

Age
Cabin
Embarked


#### Embarked

In [77]:
train_df['Embarked'].isna().sum()

2

In [78]:
train_df['Embarked'].value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [79]:
# fill null value with majority embarked city
train_df['Embarked'] = train_df['Embarked'].fillna('S')
test_df['Embarked'] = train_df['Embarked'].fillna('S')

#### Age

In [80]:
train_df['Age'].isna().sum()

177

In [81]:
train_df['Age'].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [82]:
train_df['Age'].median()

28.0

In [83]:
train_df['Age'] = train_df['Age'].fillna(train_df['Age'].median())
test_df['Age'] = test_df['Age'].fillna(test_df['Age'].median())

#### Cabin

In [84]:
train_df['Cabin'].isna().sum()

687

In [85]:
train_df['Cabin'].value_counts()

Cabin
B96 B98        4
G6             4
C23 C25 C27    4
C22 C26        3
F33            3
              ..
E34            1
C7             1
C54            1
E36            1
C148           1
Name: count, Length: 147, dtype: int64

In [86]:
imputed_train_df = train_df.drop('Cabin', axis=True)
imputed_test_df = test_df.drop('Cabin', axis=True)

In [87]:
imputed_train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1,2,W./C. 6607,23.4500,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C


## One-hot Encode Categorical Data

In [88]:
encoded_train_df = pd.get_dummies(imputed_train_df, columns=['Sex', 'Embarked'])
encoded_test_df = pd.get_dummies(imputed_test_df, columns=['Sex', 'Embarked'])

## Extract Marriage from Name

In [89]:
pattern = r'^(?P<family_name>[A-Za-z]+),\s(?P<title>Mr\.|Mrs\.|Miss\.|Rev\.)'

In [90]:
def extract_marriage_title(name):
    match = re.search(pattern, name)
    if match:
        family_name = match.group('family_name')
        title = match.group('title')
        return pd.Series([family_name, title])
    return pd.Series([None, None])

In [91]:
encoded_train_df[['family_name', 'title']] = encoded_train_df['Name'].apply(extract_marriage_title)

In [95]:
encoded_train_df['title']

0        Mr.
1       Mrs.
2      Miss.
3       Mrs.
4        Mr.
       ...  
886     Rev.
887    Miss.
888    Miss.
889      Mr.
890      Mr.
Name: title, Length: 891, dtype: object

## Remove Outliers

### 